In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
if os.path.exists('../secrets/token.json'):
  creds = Credentials.from_authorized_user_file('../secrets/token.json', SCOPES)

In [2]:
service = build('gmail', 'v1', credentials=creds)

In [3]:
results = service.users().messages().list(userId='me', labelIds=['SENT']).execute()
results.get('messages')

[{'id': '1830a892f6244a76', 'threadId': '182feb4fb2ce3e66'},
 {'id': '18303fea2d253bfb', 'threadId': '183014216c03dc44'},
 {'id': '183003a09b03f71c', 'threadId': '182feb4fb2ce3e66'},
 {'id': '182d026bdd87740e', 'threadId': '182cae82b0792f9b'},
 {'id': '182821be763be791', 'threadId': '182821be763be791'},
 {'id': '1826fb2ddfb583e3', 'threadId': '1826fb2ddfb583e3'},
 {'id': '1825e17bad28d474', 'threadId': '1825e17bad28d474'},
 {'id': '1825e13f71fea053', 'threadId': '1825e13f71fea053'},
 {'id': '1824ea3108570100', 'threadId': '1824ea3108570100'},
 {'id': '1824744ae9a6aa23', 'threadId': '1824744ae9a6aa23'},
 {'id': '1823f27849010074', 'threadId': '1823f27849010074'},
 {'id': '1822043c6747fcdd', 'threadId': '1822043c6747fcdd'},
 {'id': '182027d15ad4ebf2', 'threadId': '182027d15ad4ebf2'},
 {'id': '18202783b9d26e08', 'threadId': '18202783b9d26e08'},
 {'id': '181f988b0f3997f2', 'threadId': '181f988b0f3997f2'},
 {'id': '181f36de7cfa67c6', 'threadId': '181f36de7cfa67c6'},
 {'id': '181f36dd7e489fd

In [4]:
results.get('messages')[0].get('id')

'1830a892f6244a76'

In [5]:
# Load the actual message
top_message = results.get('messages')[0].get('id')
message = service.users().messages().get(userId="me", id=top_message).execute()
message

{'id': '1830a892f6244a76',
 'threadId': '182feb4fb2ce3e66',
 'labelIds': ['SENT'],
 'snippet': 'I should be able to make it My number is 513 550 3175 Thanks, Kyle Henderson On Sun, Sep 4 2022 at 4:44 PM, Tyler Shirley &lt;tylshir@gmail.com&gt; wrote: Want to meet at Zilker next to Barton springs',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Received',
    'value': 'from 649336022844 named unknown by gmailapi.google.com with HTTPREST; Sun, 4 Sep 2022 15:05:07 -0700'},
   {'name': 'Mime-Version', 'value': '1.0'},
   {'name': 'X-Mailer', 'value': 'Superhuman iOS 10016'},
   {'name': 'Message-ID',
    'value': '<l7nvsccu.5726d19f-8267-4792-8f49-96435414e3cd@we.are.superhuman.com>'},
   {'name': 'From', 'value': 'ona@lunch.run'},
   {'name': 'Date', 'value': 'Sun, 04 Sep 2022 22:05:06 +0000'},
   {'name': 'Subject', 'value': 'Re: Zilker Relay Team'},
   {'name': 'X-Superhuman-ID',
    'value': 'l7nvtg02.4f12f16e-ee59-41d5-9789-

In [6]:
import base64

parts = message.get('payload').get('parts')
part = None
for p in parts:
  if p.get('mimeType') == 'text/plain':
    part = p
    break
    
content = None
if part:
  data = part.get('body').get('data')
  encoded = bytes(str(data),encoding='utf-8')
  content = base64.urlsafe_b64decode(encoded).decode('utf-8')

## Quoted Content
I am trying to clear out the quoted content shit...

In [8]:
import re

lines = content.split('\n')
message = []
for line in lines:
  result = re.search(r'^On.*< \S+ > wrote:', line)
  if result:
    break
  else:
    message.append(line)

message = '\n'.join(message)
print(message)

I should be able to make it

My number is 513 550 3175

Thanks,
Kyle Henderson



# Now, lets get a thread

In [15]:
top_message = top_message = results.get('messages')[0]
top_message
thread = service.users().threads().get(userId="me", id=top_message.get("threadId")).execute()
thread

{'id': '182feb4fb2ce3e66',
 'historyId': '7836398',
 'messages': [{'id': '182feb4fb2ce3e66',
   'threadId': '182feb4fb2ce3e66',
   'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL'],
   'snippet': 'Hey guys, Kyle is down to join our team! I&#39;ve CC&#39;d him here to loop him in. Tyler, since you&#39;re the captain are you able to add him on the registration page so he gets the registration link',
   'payload': {'partId': '',
    'mimeType': 'multipart/alternative',
    'filename': '',
    'headers': [{'name': 'Delivered-To', 'value': 'ona@lunch.run'},
     {'name': 'Received',
      'value': 'by 2002:a05:600c:27d2:0:0:0:0 with SMTP id l18csp2623974wmb;        Fri, 2 Sep 2022 07:57:30 -0700 (PDT)'},
     {'name': 'X-Received',
      'value': 'by 2002:a05:6830:6315:b0:638:c7e4:b516 with SMTP id cg21-20020a056830631500b00638c7e4b516mr14740474otb.151.1662130649791;        Fri, 02 Sep 2022 07:57:29 -0700 (PDT)'},
     {'name': 'ARC-Seal',
      'value': 'i=1; a=rsa-sha256; t=1662130649; cv=no

In [26]:
# Trying to get plain text unquoted content for a message
# this way I can loop through a thread and get all the content

# TODO add some kind of error handling
def parse_message (json):
  parts = json.get('payload').get('parts')
  part = None
  # Working right now off plaintext
  for p in parts:
    if p.get('mimeType') == 'text/plain':
      part = p
      break
      
  content = None
  if part:
    data = part.get('body').get('data')
    encoded = bytes(str(data),encoding='utf-8')
    content = base64.urlsafe_b64decode(encoded).decode('utf-8')
    
  lines = content.split('\n')
  message = []
  for line in lines:
    result = re.search(r'^On.*<.*>,? wrote:', line)
    if result:
      break
    else:
      message.append(line)

  message = '\n'.join(message)
  return message
    

In [29]:
print(parse_message(thread.get('messages')[2]))

Hoi! Nice to meet you guys!

Thanks,
Kyle Henderson



In [37]:
json = service.users().messages().get(userId="me", id=top_message.get('id')).execute()
def get_previous_messages(json):
  thread = service.users().threads().get(userId="me", id=top_message.get("threadId")).execute()
  message_id = json.get('id')

  messages = thread.get('messages')
  previous_messages = []
  for message in messages:
    if message_id != message.get('id'):
      previous_messages.append(parse_message(message))
    else:
      break
      
  return previous_messages

previous_messages = get_previous_messages(json)

In [39]:
print("THREAD")
for m in previous_messages:
  print(m)
  print("=== \n")
  
print("\nRESPONSE")
print(parse_message(json))


THREAD
Hey guys,

Kyle is down to join our team! I’ve CC’d him here to loop him in.

Tyler, since you're the captain are you able to add him on the registration page so he gets the registration link?

Ryan
=== 

Hey Kyle, good to meet you!

=== 

Hoi! Nice to meet you guys!

Thanks,
Kyle Henderson

=== 

Sweet thanks I’ll send over the invite

=== 

Y’all wanna do 4-5 miles on Monday morning near ladybird?

=== 

I’m up for it

=== 

Same

Ryan
=== 

Want to meet at Zilker next to Barton springs around 8? My number’s
713-367-9063 so we can just text instead of email

=== 


RESPONSE
I should be able to make it

My number is 513 550 3175

Thanks,
Kyle Henderson

